In [134]:
import datetime
import numpy as np
import matplotlib.pyplot as plt

import akshare as ak
import pandas as pd
from dateutil.relativedelta import relativedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.offline as py

all_pe = pd.DataFrame()

# for i in tqdm(range(len(stocks))):
#     code = stocks.loc[i, "代码"]
code = '600519'
ind = ak.stock_a_lg_indicator(code)

pe = pd.Series(list(ind["pe_ttm"]), index=pd.to_datetime(ind["trade_date"]))
pb = pd.Series(list(ind["pb"]), index=pd.to_datetime(ind["trade_date"]))
ps = pd.Series(list(ind["ps_ttm"]), index=pd.to_datetime(ind["trade_date"]))

pe_df = pd.DataFrame({code: pe}).fillna(0)
pb_df = pd.DataFrame({code: pb}).fillna(0)
ps_df = pd.DataFrame({code: ps}).fillna(0)

market='sz'
if (code.startswith('00') != True):
    market ='sh'
symbol = "{}{}".format(market, code)
p = ak.stock_profit_sheet_by_quarterly_em(symbol=symbol)
eps = pd.Series(list(p['BASIC_EPS']), index=pd.to_datetime(p['REPORT_DATE']))

report_date = eps.sort_index().index[0]
start_date = report_date + pd.tseries.offsets.DateOffset(months=1 - report_date.month, days=1-report_date.day)
end_date = datetime.datetime.now()
stock_hist = ak.stock_zh_a_hist(symbol=code, period="daily", 
    start_date=start_date.strftime("%Y%m%d"), end_date=end_date.strftime("%Y%m%d"), adjust="")

cl = stock_hist["收盘"]

ts_close = pd.Series(list(cl), index=pd.to_datetime(stock_hist["日期"]))
# df_close = pd.DataFrame(ts_close, columns=["close"])
# df_close.index.name = 'Date'

# start = '2018-01'
# end = '2022-12'
# de = pe[start:end].describe()
# pe_roll = pe.rolling(min_periods=5, window=5).quantile(0.75)
# pd.concat([pe, pe_roll], axis=1).to_csv("pe_roll.csv")
# print(pe)
# print((datetime.datetime.now() + relativedelta(years=-1)).strftime("%Y-%m"))

# print(pe_roll)

# mean = de['50%']
# max = de['max']
# upper = de['75%']
# lower = de['25%']
# min = de['min']

# min_pe = pe.resample('5Y', closed="left").min()
# lower_pe = pe.resample('5Y', closed="left").quantile(0.25)
# mean_pe = pe.resample('5Y', closed="left").mean()
# upper_pe = pe.resample('5Y', closed="left").quantile(0.75)
# max_pe = pe.resample('5Y', closed="left").max()

pe_roll_days = 252 * 5
# min_pe = pe.rolling(min_periods=pe_roll_days, window=pe_roll_days).min()
# lower_pe = pe.rolling(min_periods=pe_roll_days, window=pe_roll_days).quantile(0.25)
# mean_pe = pe.rolling(min_periods=pe_roll_days, window=pe_roll_days).mean()
# upper_pe = pe.rolling(min_periods=pe_roll_days, window=pe_roll_days).quantile(0.75)
# max_pe = pe.rolling(min_periods=pe_roll_days, window=pe_roll_days).max()
mean_pe = pe.rolling(min_periods=pe_roll_days, window=pe_roll_days).mean()
std = pe.rolling(min_periods=pe_roll_days, window=pe_roll_days).std()
min_pe = mean_pe - 2 * std
lower_pe = mean_pe - std
upper_pe = mean_pe + std
max_pe = mean_pe + 2 * std

eps_sum = pd.DataFrame(eps.sort_index().rolling(min_periods=1, window=4).sum()).resample('D').ffill()
df = pd.DataFrame(ts_close, columns=['close'])

df['pe'] = pe
df['eps'] = eps
df['eps_sum'] = eps_sum
df['max_pe'] = max_pe
df['upper_pe'] = upper_pe
df['mean_pe'] = mean_pe
df['lower_pe'] = lower_pe
df['min_pe'] = min_pe
df['max'] = df['max_pe'] * df['eps_sum']
df['upper'] = df['upper_pe'] * df['eps_sum']
df['mean'] = df['mean_pe'] * df['eps_sum']
df['lower'] = df['lower_pe'] * df['eps_sum']
df['min'] = df['min_pe'] * df['eps_sum']
df.index.name = 'Date'

df.fillna(method='ffill', axis=0, inplace=True)
df_band = df.copy()
df_band.to_csv(f'band_{code}.csv')

# draw
pd.options.plotting.backend = "plotly"
# plt.style.use('ggplot')
# df_band[['close', 'max', 'upper', 'mean', 'lower', 'min']].plot(kind='line', figsize=(16, 16))

trace_list = []
items = ['close', 'max', 'upper', 'mean', 'lower', 'min']
for i in range(0, len(items)):
    trace_list.append(go.Scatter(x=df_band.index, y=df_band[items[i]], mode="lines", name=items[i]))

fig = go.Figure(trace_list)
py.plot(fig)

'temp-plot.html'

In [133]:
pe['2020-01-03':'2022-01-04'].max()

73.2944

In [93]:
import numpy as np
pe = pd.Series(list(ind["pe_ttm"]), index=pd.to_datetime(ind["trade_date"]))
pb = pd.Series(list(ind["pb"]), index=pd.to_datetime(ind["trade_date"]))
ps = pd.Series(list(ind["ps_ttm"]), index=pd.to_datetime(ind["trade_date"]))

pe_df = pd.DataFrame({code: pe}).fillna(0)
pb_df = pd.DataFrame({code: pb}).fillna(0)
ps_df = pd.DataFrame({code: ps}).fillna(0)

start = '2018-01'
end = '2022-12'
# print(pe[start:end])
mean = np.percentile(pe[start:end], 50)
std = pe[start:end].std()
mean = pe[start:end].mean()
# print(pe)
# print('mean:{}'.format(mean))
# print(pe.iloc[:,0])
# print(f'std:{std}')
# print(f'1.5 std:{mean + 1 * std}')
# print(f'3 std:{mean + 2 * std}')
# print(f'-1.5 std:{mean - 1 * std}')
# print(f'-3 std:{mean - 2 * std}')
de = pe[start:end].describe()
print(de)
print(de['25%'])
print(de['max'])
# print(29.5+5.366666*3, 29.5+5.366666*1.5)
# print(29.5-5.366666*3, 29.5-5.366666*1.5)

# print((29.5-21.4)/1.5)
# print(29.5 + (29.5-21.4)/1.5*3)
# print('1.5 std:{}', 29.5 + 1.5 * std)

count    1215.000000
mean       39.730005
std         9.690051
min        20.680700
25%        32.074300
50%        38.619900
75%        47.164300
max        73.294400
dtype: float64
32.0743
73.2944


In [ ]:
def get_pe_pb(stocks):
    q = query(valuation.day,valuation.code,valuation.pe_ratio,valuation.pb_ratio).filter(valuation.code.in_(stocks))
    rets = [get_fundamentals(q, statDate = d)for d in pd.date_range(start,end,freq='M')]
    result=pd.concat(rets)
    
    pe=result.pivot(index='day',columns='code',values='pe_ratio')
    pe.index = pd.to_datetime(pe.index, format ='%Y-%m-%d')
    pb=result.pivot(index='day',columns='code',values='pb_ratio')
    pb.index = pd.to_datetime(pb.index, format ='%Y-%m-%d')
    return pe,pb
pe,pb=get_pe_pb([stock])

In [ ]:
def draw_band(data,title,start,end):
    code=data.columns[0]
    mid=percentile(data,50)
    data["ratio"]=mid/data.iloc[:,0]
    df=get_price([code], start_date=start,end_date=end, frequency='daily', fields= 'close', skip_paused=False, fq='pre',  panel=False)
    df=df.pivot(index='time',columns='code',values='close')
    df.rename(columns={ code:"close"},inplace=True)
    
    df=df.join(data["ratio"])
    df[str(round(mid*1.3,2))+"x"]=df["close"]*df["ratio"]*1.3
    df[str(round(mid*1.15,2))+"x"]=df["close"]*df["ratio"]*1.15
    df[str(round(mid,2))+"x"]=df["close"]*df["ratio"]
    df[str(round(mid*0.85,2))+"x"]=df["close"]*df["ratio"]*0.85
    df[str(round(mid*0.7,2))+"x"]=df["close"]*df["ratio"]*0.7
    df=df.drop('ratio', axis=1)
    
    df=df.interpolate(method='linear', limit_direction='forward', axis=0)
    df.plot(figsize=(12,6),title=(ALLSTK['display_name'][code]+title+" band"))
    
    return 
draw_band(pe.copy(),"pe",start,end)
draw_band(pb.copy(),"pb",start,end)

In [140]:
3476.3533920000000 /47.43

73.2944